In [4]:
import numpy as np
import math 
import random
import time

## P20-EX1

In [8]:
f = lambda x:math.sqrt(1-x**2)
slices = [1000, 10000, 100000, 1000000]
ans = []
for slice in slices:
    sum = 0
    dx = 1.0 / slice
    for i in range(slice):
        sum += f(i * dx) * dx
    ans.append(sum * 4)
print(ans)


[3.143555466911028, 3.1417914776113167, 3.1416126164019564, 3.1415946524138207]


## P23-EX3

In [9]:
def F(a, b, c, d, n, f):
    assert a <= b and c <= d and n > 0
    sum = 0
    for i in range(n):
        x = np.random.uniform(a,b)
        y = np.random.uniform(c,d)
        if f(x) > y and y >= 0.:
            sum += 1
        elif f(x) <= y and y < 0.:
            sum -= 1
    return float(sum) / n * (b-a) * (d - c) 


In [22]:
f = lambda x:x**3-2
print (F(-1,2,-3,6,1000000,f))

-2.246265


## P36-EX5 估计整数子集 $1\sim n$的大小

In [39]:
def SetCount(X):
    S = []
    k = 0
    getUniformX = lambda :random.sample(X, 1)[0]
    a = getUniformX()
    while True:
        k += 1
        S.append(a)
        a = getUniformX()
        if a in S:
            break
    return 2*k*k/math.pi


In [47]:
N = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
ans = []
for n in N:
    X = list(np.arange(1, n+1))
    sum = 0.
    for i in range(1000):
        sum += SetCount(X)
    ans.append(sum / 1000)
print(ans)

[10.336158624160028, 122.54421322258065, 1273.099488385239, 13084.453820908178, 125759.19145613945, 1277373.8203819913, 13665333.271945456]


## P67-EX7 搜索有序链表...

In [127]:
class OrderedList():
    def __init__(self, len=10000) -> None:
        self.len = len
        self.val = []
        self.next = []
        self.pre = []
        #生成测试数据
        for i in range(len):
            self.val.append(i)
            self.next.append(i+1)
            self.pre.append(i-1)
        self.next[-1] = 0
        self.pre[0] = len - 1
        self.head = 0
        #打乱数据
        self.shuffle()


    def shuffle(self):
        for i in range(self.len):
            r = np.random.randint(self.len)
            if self.head == i:
                self.head = r
            elif self.head == r:
                self.head = i

            self.val[i], self.val[r] = self.val[r], self.val[i]

            self.next[self.pre[i]], self.next[self.pre[r]] = r, i
            self.pre[i], self.pre[r] = self.pre[r], self.pre[i]
            self.pre[self.next[i]], self.pre[self.next[r]] = r, i
            self.next[i], self.next[r] = self.next[r], self.next[i]

    #serach函数，从ptr位置开始查找，返回可能对应x的ptr和查找计数
    def __call__(self, x, ptr=None):
        if ptr is None:
            ptr = self.head
        assert ptr < self.len
        cnt = 1
        while x > self.val[ptr]:
            cnt += 1
            ptr = self.next[ptr]
            assert ptr != self.head
        return ptr, cnt

    def getRandomPtr(self):
        return np.random.randint(self.len)
        


A算法，确定性算法

In [37]:
def A(x, ordered_list):
    return ordered_list(x)

D算法，时间O(n)的概率算法

In [10]:
def D(x, ordered_list):
    ptr = ordered_list.getRandomPtr()
    y = ordered_list.val[ptr]
    if y > x:
        ptr = None
    elif y < x:
        ptr = ordered_list.next[ptr]
    else:
        return ptr, 0
    return ordered_list(x, ptr)

B算法,平均时间为O $(\sqrt n)$ 的确定算法

In [11]:
def B(x, ordered_list):
    ptr  = ordered_list.head
    _max = ordered_list.val[ptr]
    batch = int(math.sqrt(ordered_list.len))

    for i in range(batch):
        y = ordered_list.val[i]
        if _max < y and y <= x:
            ptr = i
            _max = y

    return ordered_list(x, ptr)


C算法，Sherwood版的B算法

In [118]:
def C(x, ordered_list):
    ptr  = ordered_list.head
    _max = ordered_list.val[ptr]
    batch = int(math.sqrt(ordered_list.len))
    choices = np.random.choice(ordered_list.len, size=batch, replace=False)

    for choice in choices:
        y = ordered_list.val[choice]
        if _max < y and y <= x:
            ptr = choice
            _max = y

    return ordered_list(x, ptr)

性能测试

In [130]:
len = 10000
test = OrderedList(len)
test_times = 5000

worst = {"A":0, "B":0, "C":0, "D":0}
average = {"A":0., "B":0., "C":0., "D":0.}
alg_s = {"A":A, "B":B, "C":C, "D":D}

for name, alg in alg_s.items():
    for i in range(test_times):
        x = np.random.randint(len)
        ptr, cnt = alg(x, test)
        assert test.val[ptr] == x
        average[name] += cnt
        if cnt > worst[name]:
            worst[name] = cnt
    average[name] /= test_times


print(worst)
print(average)

{'A': 10000, 'B': 483, 'C': 892, 'D': 9785}
{'A': 4950.6982, 'B': 91.284, 'C': 98.3144, 'D': 3286.0576}
